<center><p float="center">
  <img src="https://upload.wikimedia.org/wikipedia/commons/e/e9/4_RGB_McCombs_School_Brand_Branded.png" width="300" height="100"/>
  <img src="https://mma.prnewswire.com/media/1458111/Great_Learning_Logo.jpg?p=facebook" width="200" height="100"/>
</p></center>

<center><font size=10>Generative AI for Business Applications</center></font>
<center><font size=6>Large Language Models & Prompt Engineering - Week 2</center></font>

<center><p float="center">
  <img src="https://cdn.pixabay.com/photo/2017/07/24/04/23/technical-support-2533526_1280.png" width="480"/>
</p></center>

<center><font size=5>Support Ticket Categorization</center></font>

# Problem Statement

## Business Context

In today’s dynamic business landscape, organizations recognize the critical role of customer feedback in shaping products and services. Effectively leveraging this feedback enhances customer experiences, drives growth, and fosters long-term relationships. For Product Managers and Analysts, staying aligned with the voice of the customer is a strategic imperative.

While organizations receive vast amounts of customer feedback and support tickets, the challenge lies in managing and utilizing this data effectively. A structured approach is essential, one that identifies key issues, prioritizes efficiently, and allocates resources wisely. Implementing a Support Ticket Categorization system is a powerful strategy to meet these needs; without it, teams may struggle to respond promptly to critical issues, leading to decreased customer satisfaction and engagement.

## Objective

The primary goals of the proposed support ticket categorization system are accurate classification, enabling a tagging mechanism, prioritization based on customer sentiment, and automated first response generation.

The implementation of such a support ticket categorization system will empower the organization to respond proactively to customer feedback, ultimately leading to improved customer experiences and stronger, more enduring relationships with their client base, optimize resource allocation to address high-impact issues, and drive both growth and customer loyalty.

## Data Description

The dataset contains the following two columns:

* **support\_tick\_id**: A unique identifier assigned to each support ticket.
* **support\_ticket\_text**: The text content describing the issue reported in the support ticket.

# Installing and Importing Necessary Libraries and Dependencies

In [ ]:
!pip install -q transformers==4.53.2 \
                  accelerate==1.8.1 \
                  openai==1.96.1 \
                  bitsandbytes==0.46.1

Note:
- After running the above cell, kindly restart the runtime (for Google Colab) or notebook kernel (for Jupyter Notebook), and run all cells sequentially from the next cell.
- On executing the above line of code, you might see a warning regarding package dependencies. This error message can be ignored as the above code ensures that all necessary libraries and their dependencies are maintained to successfully execute the code in this notebook.

**Prompt:**

<font size=3 color="#4682B4"><b>I want to analyze the provided CSV data and work with AI models to understand the support tickets. Help me import the necessary Python libraries to:

1. Read and manipulate the data</ul>
2. Work with JSON data
3. Working with system enviroment
4. Connect to OpenAI models
5. Use models from Hugging Face with AutoTokenizer and AutoModelForCausalLM

</font>

<font size=3 color="#4682B4"><b>
These libraries will help us load the data, connect with AI models, and prepare for further steps in the project.

</font>

In [ ]:
import pandas as pd
import json
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
from openai import OpenAI

# Loading the Data

***Prompt***:

<font size=3 color="#4682B4"><b> Mount the Google Drive
</font>

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

***Prompt***:

<font size=3 color="#4682B4"><b> Load the CSV file named "support_ticket_data" and store it in the variable data.
</font>

In [ ]:
# Load your CSV file
data = pd.read_csv("/content/support_ticket_data.csv")

# Data Overview

***Prompt***:

<font size=3 color="#4682B4"><b> Display the number of rows and columns in the `data`.
</font>

In [ ]:
data.shape

* There are 21 rows and 2 columns.

***Prompt***:

<font size=3 color="#4682B4"><b> Display the first 5 rows of the `data`.
</font>

In [ ]:
data.head()

We create a copy of the original DataFrame to ensure that we always have the original support ticket data

In [ ]:
df=data.copy()

# Model Loading

We'll be using two language models:

1. `mistralai/Mistral-7B-Instruct-v0.1` - an open-source model from Hugging Face.
2. OpenAI's GPT model - accessed using an API key.

Before we start using these models, we need to set up and securely load our API tokens into the environment.  
This ensures authenticated access to both Hugging Face and OpenAI services.


## Setting Up Hugging Face Token and OpenAI API Key

Set Your OpenAI and Hugging face keys in the `config.json` file provided.

1. Replace `"your-hugging-face-token"` with your actual Hugging Face key.
2. Replace `"your_api_key_here"` with your actual OpenAI API key.
3. Replace `"your_base_url_here"` with your OpenAI base URL



In [ ]:
import json

# Load from your config.json (update the path if needed)
with open("Config.json", "r") as f:
    config = json.load(f)

# Extract the token
HF_TOKEN = config.get('HF_TOKEN')
OPENAI_API_KEY = config.get("OPENAI_API_KEY")
OPENAI_API_BASE = config.get("OPENAI_API_BASE")

## Loading Mistral Model

**NOTE**

1. We’re loading the entire model onto the local machine, which might take some time to initialize. To optimize this, we use 8-bit loading to reduce memory usage and speed up inference without significantly impacting performance.

2. Before loading the model, you must first agree to its terms and conditions on Hugging Face. To do this, search for the model on the Hugging Face website, review its license or usage restrictions, and click “Agree and Access” to enable programmatic access via code.


***Prompt***:

<font size=3 color="#4682B4"><b> Load the `mistralai/Mistral-7B-Instruct-v0.1` from hugging face using 8-bit quantization.

</font>

In [ ]:
import torch

model_id = "mistralai/Mistral-7B-Instruct-v0.1"

tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_8bit=True,                 # Load the model with 8-bit quantization
    torch_dtype=torch.float16,         # Use 16-bit floats on GPU
    device_map="auto",                 # Automatically assign GPU or CPU
    token=HF_TOKEN                     # Hugging Face token for access
)

* `load_in_8bit=True`: Loads the model using 8-bit quantization to save memory.
* `torch_dtype=torch.float16`: Uses half-precision (16-bit) floats for faster computation on GPU.
* `device_map="auto"`: Automatically places model layers across available devices.


The Hugging Face model is now ready. Let’s test it on an example input.

***Prompt***:

<font size=3 color="#4682B4"><b> Ask the Mistral model: What is the capital of France?
</font>

In [ ]:
# Define the prompt (question)
prompt = "### Question: What is the capital of France?\n### Answer:"

# Tokenize input
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate response
outputs = model.generate(**inputs)

# Decode and print the output
response = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(response)

As we can see, the model is returning results successfully.

Let’s define a function that takes a `prompt` and a `query` as inputs and returns the model’s output.  

- This will make it easier to reuse the model across different inputs.

***Prompt***:

<font size=3 color="#4682B4"><b> Create a function that accepts a prompt and query, and returns the response generated by the Mistral model.
</font>

In [ ]:
def query_mistral(prompt, query):
    """
    Queries the Mistral model with a given prompt and query.

    Args:
        prompt (str): The prompt for the model.
        query (str): The query to be answered by the model.

    Returns:
        str: The model's response.
    """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": query}
    ]
    inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to(model.device)

    pad_token_id = tokenizer.pad_token_id or tokenizer.eos_token_id

    attention_mask = (inputs != pad_token_id).long()

    outputs = model.generate(
        inputs,
        attention_mask=attention_mask,
        max_new_tokens=100,  # Adjust as needed
        do_sample=True,
        temperature=0.7,     # Adjust as needed
        top_p=0.9,           # Adjust as needed
        pad_token_id=pad_token_id  # Prevents warning
    )

    # Decode and print the output, skipping the input tokens
    response = tokenizer.decode(outputs[0][inputs.shape[-1]:], skip_special_tokens=True)
    return response


In the code snippet defined above, the following components are used:

1. `tokenizer.apply_chat_template()`: This method converts the `messages` list into a single formatted string (e.g., adding special tokens or chat-style formatting), and tokenizes it into a tensor using PyTorch (`return_tensors="pt"`). The `.to(model.device)` part ensures the tokenized input is moved to the same device as the model (like a GPU or CPU).

2. `pad_token_id`: This variable is assigned the padding token ID used by the tokenizer. If the tokenizer does not explicitly define a `pad_token_id`, it falls back to the `eos_token_id` (end-of-sequence token). This is needed to handle padding properly during attention and generation.

3. `attention_mask:` This creates a mask that tells the model which tokens should be attended to (represented by 1) and which should be ignored (usually padding tokens, represented by 0). It ensures the model focuses only on valid input tokens during processing.

In the `generate()` function defined above, the following arguments are used:

1. `max_new_tokens`: This parameter determines the maximum length of the generated sequence. In the provided code, max_new_tokens is set to 100, which means the generated sequence should not exceed 100 tokens.

2. `temperature`: The temperature parameter controls the level of randomness in the generation process. A higher temperature (e.g., closer to 1) makes the output more diverse and creative but potentially less focused, while a lower temperature (e.g., close to 0) produces more deterministic and focused but potentially repetitive outputs. In the code, temperature is set to 0.7, indicating a very low temperature and, consequently, a more deterministic sampling.

3. `do_sample`: This is a boolean parameter that determines whether to use sampling during generation (do_sample=True) or use greedy decoding (do_sample=False). When set to True, as in the provided code, the model samples from the distribution of predicted tokens at each step, introducing randomness in the generation process.

4. `top_p`: Controls how many top probable tokens to consider during generation. If set to 0.9, it samples from the smallest set of tokens whose combined probability is at least 90%, balancing creativity and coherence.


## Loading OpenAI model

***Prompt***:

<font size=3 color="#4682B4"><b> Create an OpenAI client
</font>

In [ ]:
openai_client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_API_BASE)

OpenAI model is now ready. Let’s test it on an example input.

***Prompt***:

<font size=3 color="#4682B4"><b> Ask the OpenAI model: What is the capital of France?
</font>

In [ ]:
# prompt: Ask the OpenAI model: What is the capital of France?

query = "What is the capital of France?"

response = openai_client.chat.completions.create(
    model="gpt-3.5-turbo",  # Or another suitable OpenAI model
    messages=[
        {"role": "user", "content": query}
    ],
    max_tokens=100
)

response.choices[0].message.content


The model is returning results successfully.

So, let’s define a function that takes a `prompt` and a `query` as inputs and returns the model’s output.

***Prompt***:

<font size=3 color="#4682B4"><b> Create a function that accepts a prompt and query, and returns the response generated by the OpenAI model.
</font>

In [ ]:
# prompt: Create a function that accepts a prompt and query, and returns the response generated by the OpenAI model.

def query_openai(prompt, query):
    """
    Queries the OpenAI model with a given prompt and query.

    Args:
        prompt (str): The prompt for the model.
        query (str): The query to be answered by the model.

    Returns:
        str: The model's response.
    """
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": query}
    ]
    response = openai_client.chat.completions.create(
        model="gpt-3.5-turbo",  # Or another suitable OpenAI model
        messages=messages,
        max_tokens=500  # Adjust max_tokens as needed
    )
    return response.choices[0].message.content

# Example usage:
# prompt_text = "You are a helpful assistant."
# query_text = "Explain the process of ticket categorization."
# openai_response = query_openai(prompt_text, query_text)
# print(openai_response)


# Ticket Categorization and Response Generation

Let’s take a sample from the support tickets to observe how the model performs on each task.


***Prompt***:

<font size=3 color="#4682B4"><b>Take the first "support_ticket_text" from the data and store it in the variable sample_ticket.
</font>

In [ ]:
sample_ticket=data['support_ticket_text'][0]

## Task 1: Ticket Categorization using Zero Shot Prompting

Our first task is to classify the support ticket into a predefined category. We will write a prompt that instructs the model to return the category in **JSON format**, so that we receive the output as structured data that’s easy to parse and store.

In [ ]:
classification_prompt = """
 You are a technical assistant. Classify the support ticket based on the Support Ticket Text presented in the input into the following categories and not any other.
    - Technical issues
    - Hardware issues
    - Data recovery
Return only a structured JSON output in the following format:
{"Category": "category_prediction"}
"""

### Using Mistral

***Prompt***:

<font size=3 color="#4682B4"><b> Define a function named classify_ticket_mistral that takes the classification prompt and the support ticket text as input, gets the result from query_mistral function, and returns the result in a JSON format.
</font>

In [ ]:
def classify_ticket_mistral(prompt, query):
    """
    Classifies a support ticket using the OpenAI model and returns the result in JSON format.

    Args:
        prompt (str): The classification prompt for the model.
        query (str): The support ticket text to be classified.

    Returns:
        dict: A dictionary containing the classification result, or None if classification fails.
    """
    try:
        response_text = query_mistral(prompt, query)
        # Attempt to parse the response text as JSON
        classification_result = json.loads(response_text)
        return classification_result
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from OpenAI response: {e}")
        print(f"Raw OpenAI response: {response_text}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None



In [ ]:
classify=classify_ticket_mistral(classification_prompt,sample_ticket)
print(sample_ticket)
print(classify)

> Ticket text : My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.

> Category : Technical Issue

### Using OpenAI

***Prompt***:

<font size=3 color="#4682B4"><b> Define a function named classify_ticket_openai that takes the classification prompt and the support ticket text as input, gets the result from query_openai function, and returns the result in a JSON format.
</font>

In [ ]:
def classify_ticket_openai(prompt, query):
    """
    Classifies a support ticket using the OpenAI model and returns the result in JSON format.

    Args:
        prompt (str): The classification prompt for the model.
        query (str): The support ticket text to be classified.

    Returns:
        dict: A dictionary containing the classification result, or None if classification fails.
    """
    try:
        response_text = query_openai(prompt, query)
        # Attempt to parse the response text as JSON
        classification_result = json.loads(response_text)
        return classification_result
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from OpenAI response: {e}")
        print(f"Raw OpenAI response: {response_text}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None



In [ ]:
classify=classify_ticket_openai(classification_prompt,sample_ticket)
print(sample_ticket)
print(classify)

> Ticket Text : My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.

> Category : Technical Issue

Both models generate results in the desired format. However, we'll proceed with OpenAI, as its API-based setup provides slightly faster responses.

***Prompt***:

<font size=3 color="#4682B4"><b>Generate the category for each support_ticket_text in the DataFrame using the classify_ticket_openai function, and store the result in a new column.

</font>

In [ ]:
# Apply the classification function to each row in the DataFrame
df['Category'] = df['support_ticket_text'].apply(lambda x: classify_ticket_openai(classification_prompt, x)['Category'] if classify_ticket_openai(classification_prompt, x) else None)

In [ ]:
df

## Task 2: Creating Tags using Few Shot Prompting

For this task, we will use few-shot prompting instead of zero-shot prompting because it provides the model with concrete examples, helping it understand how to extract and structure the information accurately.
- Zero-shot prompting may result in inconsistent formats, incorrect category
mapping, or misinterpretation of impact levels due to a lack of context and guidance.

In [ ]:
metadata_prompt = """
You are an intelligent assistant that extracts structured metadata from technical support queries.
Analyze the query and extract the following information:

* Device (e.g., Laptop, Phone, Router, etc.)
* Problem Type (e.g., Not Turning On, Lost Internet, Deleted Files)
* User Impact - Estimate based on how severely the issue affects the user's ability to continue working or using the device:

    - * Major: The user cannot proceed with work at all.
    - * Moderate: The user is impacted but may have a workaround.
    - * Minor: The issue is present but does not significantly hinder usage.

Use the following examples as guidance.

Query Text: My phone battery is draining rapidly even on battery saver mode. I barely use it and it drops 50% in a few hours.
Output: {"Device": "Phone", "Problem Type": "Battery Draining", "User Impact": "Minor"}

Query Text: I accidentally deleted a folder containing all project files. Please help me recover it.
Output: {"Device": "Laptop", "Problem Type": "Deleted Files", "User Impact": "Major"}

Query Text: My router is not working.
Output: {"Device": "Router", "Problem Type": "Lost Internet", "User Impact": "Moderate"}

Return the final output only in a valid JSON format without any additional explanation.
"""

### Using Mistral

***Prompt***:

<font size=3 color="#4682B4"><b> Define a function named extract_metadata_mistral that takes the metadata prompt and query as input get the result from query_mistral function return the result in JSON format.
</font>

In [ ]:
def extract_metadata_mistral(prompt, query):
    """
    Extracts metadata from a support ticket using the OpenAI model and returns the result in JSON format.

    Args:
        prompt (str): The metadata extraction prompt for the model.
        query (str): The support ticket text to extract metadata from.

    Returns:
        dict: A dictionary containing the extracted metadata, or None if extraction fails.
    """
    try:
        response_text = query_mistral(prompt, query)
        # Attempt to parse the response text as JSON
        metadata_result = json.loads(response_text)
        return metadata_result
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from OpenAI response: {e}")
        print(f"Raw OpenAI response: {response_text}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [ ]:
metadata=extract_metadata_mistral(metadata_prompt,sample_ticket)
print(sample_ticket)
print(metadata)

> Ticket text : My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.

> Category : Technical Issue

> Device : Laptop

> Problem Type : Lost Internet

> User Impace : Major

**Observation:**

* The issue described is "Slowed Internet" due to frequent disconnections, not a complete loss; the problem likely lies with the router, not the laptop.
* The model incorrectly tagged the **Device** as "Laptop" and **Problem Type** as "Lost Internet."

Let's try to generate metadata using OpenAI.


### Using Openai

***Prompt***:

<font size=3 color="#4682B4"><b> Define a function named extract_metadata_openai that takes the metadata prompt and query as input get the result from query_openai function return the result in JSON format.
</font>

In [ ]:
def extract_metadata_openai(prompt, query):
    """
    Extracts metadata from a support ticket using the OpenAI model and returns the result in JSON format.

    Args:
        prompt (str): The metadata extraction prompt for the model.
        query (str): The support ticket text to extract metadata from.

    Returns:
        dict: A dictionary containing the extracted metadata, or None if extraction fails.
    """
    try:
        response_text = query_openai(prompt, query)
        # Attempt to parse the response text as JSON
        metadata_result = json.loads(response_text)
        return metadata_result
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from OpenAI response: {e}")
        print(f"Raw OpenAI response: {response_text}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [ ]:
metadata=extract_metadata_openai(metadata_prompt,sample_ticket)
print(sample_ticket)
print(metadata)

> Ticket text : My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.

> Category : Technical Issue

> Device : Router

> Problem Type : Slow Internet and Frequent Disconnections

> User Impace : Major

**Observation:**

* OpenAI is generating the correct metadata based on the ticket description.
* Therefore, we will proceed with OpenAI for further ticket categorization.

***Prompt***:

<font size=3 color="#4682B4"><b> Generate the metadata for each support_ticket_text in the DataFrame using extract_metadata_openai, store it in a new column, and extract individual fields into separate columns.
</font>

In [ ]:
# Generate metadata for each support_ticket_text and store it in a new column
df['Metadata'] = df['support_ticket_text'].apply(lambda x: extract_metadata_openai(metadata_prompt, x))

# Extract individual metadata fields into separate columns
df['Device'] = df['Metadata'].apply(lambda x: x.get('Device') if x else None)
df['Problem Type'] = df['Metadata'].apply(lambda x: x.get('Problem Type') if x else None)
df['User Impact'] = df['Metadata'].apply(lambda x: x.get('User Impact') if x else None)

In [ ]:
df

## Task 3: Predicting Priority using Chain of Thought Prompting

For this task, Chain-of-Thought (CoT) prompting is crucial because assigning support ticket priority involves understanding several subtle cues like urgency, usability, and user sentiment.

- CoT allows the model to reason step-by-step through these factors, leading to
more accurate and context-aware decisions. Without this, zero-shot prompts may miss key details and result in incorrect or inconsistent priority levels.

In [ ]:
priority_prompt ="""
You are an intelligent assistant that determines the priority level of a support ticket.

For any given ticket, follow this step-by-step reasoning process to assign the correct priority level: Low, Medium, High.

Step-by-step Evaluation:

Is the device or service completely unusable?

Is the issue blocking critical or time-sensitive work?

Is there a specific deadline or urgency mentioned by the user?

Does the user mention partial functionality or ongoing work?

Is the tone or language expressing frustration or emergency?

After evaluating all the above steps, decide the most appropriate priority level based on the impact and urgency.

Return the final output only in a valid JSON format as mentioned below without any additional explanation:
{"priority": "High"}
"""

***Prompt***:

<font size=3 color="#4682B4"><b> Define a function that takes the priority prompt, problem type, user Impact, and query as input get the result from query_openai function return the result in JSON format.
</font>

In [ ]:
def predict_priority(prompt, query, problem_type, user_impact):
    """
    Predicts the priority of a support ticket using the OpenAI model and returns the result in JSON format.

    Args:
        prompt (str): The priority prediction prompt for the model.
        query (str): The support ticket text to predict the priority for.
        problem_type (str): The extracted problem type.
        user_impact (str): The extracted user impact.

    Returns:
        dict: A dictionary containing the predicted priority, or None if prediction fails.
    """
    try:
        # Include problem_type and user_impact in the query sent to the model
        full_query = f"""
        Support Ticket: {query}
        Problem Type: {problem_type}
        User Impact: {user_impact}

        Based on the support ticket, problem type, and user impact, predict the priority: Low, Medium, High.
        Return only a structured JSON output in the following format:
        {{"priority": "priority_prediction"}}
        """
        response_text = query_openai(prompt, full_query)
        priority_result = json.loads(response_text)
        return priority_result
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON from OpenAI response: {e}")
        print(f"Raw OpenAI response: {response_text}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [ ]:
priority = predict_priority(priority_prompt, sample_ticket,metadata['Problem Type'],metadata['User Impact'])
print(sample_ticket)
priority

> Ticket text : My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.

> Category : Technical Issue

> Device : Router

> Problem Type : Slow Internet Connection

> User Impact : Major

> Priority: High

***Prompt***:

<font size=3 color="#4682B4"><b> Generate the priotity for each support_ticket_text in the df and store it in a new column.
</font>

In [ ]:
# prompt: Generate the priotity for each support_ticket_text in the df and store it in a new column
# Generate the priority for each support_ticket_text and store it in a new column
df['Priority'] = df.apply(lambda row: predict_priority(priority_prompt, row['support_ticket_text'], row['Problem Type'], row['User Impact'])['priority'] if predict_priority(priority_prompt, row['support_ticket_text'], row['Problem Type'], row['User Impact']) else None, axis=1)

In [ ]:
df

## Task 4 - Creating a Draft Response using Chain of Thought Prompting

We will continue using Chain-of-Thought (CoT) prompting for this task because generating a thoughtful, empathetic response requires the model to reason through multiple elements, such as the user's concern, the issue type, priority-based urgency, and appropriate tone.
- CoT helps the model integrate all these cues step-by-step to produce a relevant and human-like reply.

In [ ]:
response_prompt = """
You are provided with a support ticket's text along with its Category, Tags, and assigned Priority level.

Follow these steps before generating your final response:

1. Analyze the ticket text to understand the customer's sentiment and main concern.
2. Identify the issue type using the provided Category and Tags.
3. Determine the appropriate ETA based on the Priority level.
4. Compose a short, empathetic response that reassures the customer, acknowledges their concern, and includes the ETA.

Ensure the final response:

1. Is under 50 words
2. Has a polite and empathetic tone
3. Addresses the issue clearly

Return only the final response to the customer. Do not include your reasoning steps in the output.
"""

***Prompt***:

<font size=3 color="#4682B4"><b> Define a function that takes the `response_prompt`, `query`, `category`, `metadata_tags`, and `priority` as inputs. Pass `response_prompt` as the system prompt, and combine the remaining inputs (`query`, `category`, `metadata_tags`, and `priority`) into a single message. Pass this message to the `query_openai` function and return the result.
</font>

In [ ]:
# prompt: Define a function that takes response_prompt, query, category, metadata_tags, and priority as inputs. Pass response_prompt as the system prompt, and combine the remaining inputs (query, category, metadata_tags, and priority) into a single message. Pass this message to the query_openai function and return the result.
def generate_response(response_prompt, query, category, metadata_tags, priority):
    """
    Generates a draft response for a support ticket using the OpenAI model.

    Args:
        response_prompt (str): The prompt for generating the response.
        query (str): The original support ticket text.
        category (str): The predicted category of the ticket.
        metadata_tags (dict): The extracted metadata tags (Device, Problem Type, User Impact).
        priority (str): The predicted priority of the ticket.

    Returns:
        str: The generated response text, or None if response generation fails.
    """
    # Combine the inputs into a single message for the model
    user_message = f"""
    Support Ticket: {query}
    Category: {category}
    Metadata Tags: {metadata_tags}
    Priority: {priority}
    """

    try:
        # Pass the combined message to the query_openai function
        response_text = query_openai(response_prompt, user_message)
        return response_text
    except Exception as e:
        print(f"An unexpected error occurred during response generation: {e}")
        return None

# Example usage (assuming you have values for query, category, metadata_tags, priority):
# sample_query = df['support_ticket_text'][0]
# sample_category = df['Category'][0]
# sample_metadata_tags = df['Metadata'][0]
# sample_priority = df['Priority'][0]

# draft_response = generate_response(response_prompt, sample_query, sample_category, sample_metadata_tags, sample_priority)
# print(draft_response)

In [ ]:
sample_query = df['support_ticket_text'][0]
sample_category = df['Category'][0]
sample_metadata_tags = df['Metadata'][0]
sample_priority = df['Priority'][0]

draft_response = generate_response(response_prompt, sample_query, sample_category, sample_metadata_tags, sample_priority)
print(draft_response)

> Ticket text : My internet connection has significantly slowed down over the past two days, making it challenging to work efficiently from home. Frequent disconnections are causing major disruptions. Please assist in resolving this connectivity issue promptly.

> Category : Technical Issue

> Device : Router

> Problem Type : Slow Internet Connection

> User Impact : Major

> Priority : High

> Response : I understand the urgency of your situation. Our team will prioritize resolving your slow internet and disconnection issues. We aim to have this addressed within the next 24 hours to improve your work efficiency. Thank you for your patience.


***Prompt***:

<font size=3 color="#4682B4"><b> Generate the draft_response for each support_ticket_text in the df and store it in a new column.
</font>

In [ ]:
df['draft_response'] = df.apply(lambda row: generate_response(response_prompt, row['support_ticket_text'], row['Category'], row['Metadata'], row['Priority']), axis=1)

In [ ]:
df

## Reviewing Model Output

Now that we have converted all the support tickets into structured data, let's review a few tickets along with their classifications.


***Prompt***:

<font size=3 color="#4682B4"><b> Create a function that takes a support ticket ID as input and displays the details extracted from the model on a separate line.
</font>

In [ ]:
def display_ticket_details(ticket_id, dataframe):
  """
  Displays all details of a support ticket from a DataFrame, with each detail on a new line.

  Args:
    ticket_id (str): The ID of the support ticket to display.
    dataframe (pd.DataFrame): The DataFrame containing the support ticket data.
  """
  ticket_details = dataframe[dataframe['support_tick_id'] == ticket_id]

  if not ticket_details.empty:
    for column in ticket_details.columns:
      print(f"{column}: {ticket_details.iloc[0][column]}")
  else:
    print(f"Ticket with ID {ticket_id} not found.")

### Ticket ID: ST2023-009

In [ ]:
# Example usage:
display_ticket_details('ST2023-009', df)

**Observations:**

- The system correctly identifies the device as *Router* and the problem type as *Weak Wi-Fi Signal*, matching the ticket description.
- The impact is rightly marked as *Moderate*, as the issue is persistent and affects usability.
- The draft response is polite, empathetic, and includes a clear resolution timeline, which helps manage user expectations.


### Ticket ID: ST2023-018

In [ ]:
# Example usage:
display_ticket_details('ST2023-018', df)

**Observations:**

- The system accurately detects the *Device* as **Laptop** and the *Problem Type* as **Liquid Damage**, aligning with the ticket text.
- The *User Impact* is rightly set as **Major**, given the device won’t turn on, and *Priority* is appropriately marked as **High**.
- The draft response is empathetic and assures immediate action with a ETA 2-3 days, which builds user trust.


### Ticket ID: ST2023-023

In [ ]:
# Example usage:
display_ticket_details('ST2023-023', df)

**Observations:**

- The *Device* is correctly identified as **USB Drive**, and the *Problem Type* as **Formatted Drive**, matching the user’s concern.
- The *User Impact* is rightly marked **Major** due to potential data loss, with *Priority* set to **High**.
- The draft reply is empathetic and ensures quick action with a clear resolution timeline of **24 hours**, reassuring the user.


# Deployment

Now, we deploy the system using Hugging Face Spaces with Streamlit.

> **Note:** This is purely for demonstration purposes, so we won't dive deep into the code here.

## Streamlit on Hugging Face

### app.py

**NOTE**: You can use this prompt to generate the code for buidling the Streamlit app. However, a few minor adjustments might be needed afterwards.

***Prompt***:

<font size=3 color="#4682B4"><b> Create a complete Streamlit app that includes all required functions and prompts, uses the OpenAI API to fetch results, and displays all the final outputs at the end.
</font>

In [ ]:
# %%writefile app.py

# import streamlit as st
# import pandas as pd
# import json
# import os
# from openai import OpenAI

# # Load OpenAI API key and base URL from Colab secrets
# try:
#     OPENAI_API_KEY = os.environ.get("API_KEY")
#     OPENAI_API_BASE = os.environ.get("API_BASE")
#     openai_client = OpenAI(api_key=OPENAI_API_KEY, base_url=OPENAI_API_BASE)
# except Exception as e:
#     st.error(f"Error loading OpenAI credentials: {e}")
#     st.stop()


# # Define the functions for categorization, metadata extraction, priority prediction, and response generation
# def query_openai(prompt, query):
#     """
#     Queries the OpenAI model with a given prompt and query.
#     Args:
#         prompt (str): The prompt for the model.
#         query (str): The query to be answered by the model.
#     Returns:
#         str: The model's response.
#     """
#     messages = [
#         {"role": "system", "content": prompt},
#         {"role": "user", "content": query}
#     ]
#     response = openai_client.chat.completions.create(
#         model="gpt-3.5-turbo",  # Or another suitable OpenAI model
#         messages=messages,
#         max_tokens=100  # Adjust max_tokens as needed
#     )
#     return response.choices[0].message.content

# def classify_ticket(prompt, query):
#     """
#     Classifies a support ticket using the OpenAI model and returns the result in JSON format.
#     Args:
#         prompt (str): The classification prompt for the model.
#         query (str): The support ticket text to be classified.
#     Returns:
#         dict: A dictionary containing the classification result, or None if classification fails.
#     """
#     try:
#         response_text = query_openai(prompt, query)
#         # Attempt to parse the response text as JSON
#         classification_result = json.loads(response_text)
#         return classification_result
#     except json.JSONDecodeError as e:
#         st.error(f"Error decoding JSON from OpenAI response: {e}")
#         st.text(f"Raw OpenAI response: {response_text}")
#         return None
#     except Exception as e:
#         st.error(f"An unexpected error occurred during classification: {e}")
#         return None

# def extract_metadata(prompt, query):
#     """
#     Extracts metadata from a support ticket using the OpenAI model and returns the result in JSON format.
#     Args:
#         prompt (str): The metadata extraction prompt for the model.
#         query (str): The support ticket text to extract metadata from.
#     Returns:
#         dict: A dictionary containing the extracted metadata, or None if extraction fails.
#     """
#     try:
#         response_text = query_openai(prompt, query)
#         # Attempt to parse the response text as JSON
#         metadata_result = json.loads(response_text)
#         return metadata_result
#     except json.JSONDecodeError as e:
#         st.error(f"Error decoding JSON from OpenAI response: {e}")
#         st.text(f"Raw OpenAI response: {response_text}")
#         return None
#     except Exception as e:
#         st.error(f"An unexpected error occurred during metadata extraction: {e}")
#         return None

# def predict_priority(prompt, query, problem_type, user_impact):
#     """
#     Predicts the priority of a support ticket using the OpenAI model and returns the result in JSON format.
#     Args:
#         prompt (str): The priority prediction prompt for the model.
#         query (str): The support ticket text to predict the priority for.
#         problem_type (str): The extracted problem type.
#         user_impact (str): The extracted user impact.
#     Returns:
#         dict: A dictionary containing the predicted priority, or None if prediction fails.
#     """
#     try:
#         # Include problem_type and user_impact in the query sent to the model
#         full_query = f"""
#         Support Ticket: {query}
#         Problem Type: {problem_type}
#         User Impact: {user_impact}
#         Based on the support ticket, problem type, and user impact, predict the priority: Low, Medium, High, or Urgent.
#         Return only a structured JSON output in the following format:
#         {{"priority": "priority_prediction"}}
#         """
#         response_text = query_openai(prompt, full_query)
#         priority_result = json.loads(response_text)
#         return priority_result
#     except json.JSONDecodeError as e:
#         st.error(f"Error decoding JSON from OpenAI response: {e}")
#         st.text(f"Raw OpenAI response: {response_text}")
#         return None
#     except Exception as e:
#         st.error(f"An unexpected error occurred during priority prediction: {e}")
#         return None

# def generate_response(response_prompt, query, category, metadata_tags, priority):
#     """
#     Generates a draft response for a support ticket using the OpenAI model.
#     Args:
#         response_prompt (str): The prompt for generating the response.
#         query (str): The original support ticket text.
#         category (str): The predicted category of the ticket.
#         metadata_tags (dict): The extracted metadata tags (Device, Problem Type, User Impact).
#         priority (str): The predicted priority of the ticket.
#     Returns:
#         str: The generated response text, or None if response generation fails.
#     """
#     # Combine the inputs into a single message for the model
#     user_message = f"""
#     Support Ticket: {query}
#     Category: {category}
#     Metadata Tags: {metadata_tags}
#     Priority: {priority}
#     """

#     try:
#         # Pass the combined message to the query_openai function
#         response_text = query_openai(response_prompt, user_message)
#         return response_text
#     except Exception as e:
#         st.error(f"An unexpected error occurred during response generation: {e}")
#         return None

# # Define the prompts
# classification_prompt = """
#  You are a technical assistant. Classify the support ticket based on the Support Ticket Text presented in the input into the following categories and not any other.
#       - Technical issues
#       - Hardware issues
#       - Data recovery
#   Return only a structured JSON output in the following format:
#   {"Category": "category_prediction"}
# """

# metadata_prompt = f"""
# You are an intelligent assistant that extracts structured metadata from technical support queries.
# Analyze the query and extract the following information:

# * Device (e.g., Laptop, Phone, Router, etc.)
# * Problem Type (e.g., Not Turning On, Lost Internet, Deleted Files)
# * User Impact - Estimate based on how severely the issue affects the user's ability to continue working or using the device:

#     - * Major: The user cannot proceed with work at all.
#     - * Moderate: The user is impacted but may have a workaround.
#     - * Minor: The issue is present but does not significantly hinder usage.

# Use the following examples as guidance.

# Query Text: My phone battery is draining rapidly even on battery saver mode. I barely use it and it drops 50% in a few hours.
# Output: {"Device": "Phone", "Problem Type": "Battery Draining", "User Impact": "Minor"}

# Query Text: I accidentally deleted a folder containing all project files. Please help me recover it.
# Output: {"Device": "Laptop", "Problem Type": "Deleted Files", "User Impact": "Major"}

# Query Text: My router is not working.
# Output: {"Device": "Router", "Problem Type": "Lost Internet", "User Impact": "Moderate"}

# Return the final output only in a valid JSON format without any additional explanation.
# """

# priority_prompt ="""
# You are an intelligent assistant that determines the priority level of a support ticket.
# For any given ticket, follow this step-by-step reasoning process to assign the correct priority level: Low, Medium, High.
# Step-by-step Evaluation:
# Is the device or service completely unusable?
# Is the issue blocking critical or time-sensitive work?
# Is there a specific deadline or urgency mentioned by the user?
# Does the user mention partial functionality or ongoing work?
# Is the tone or language expressing frustration or emergency?
# After evaluating each step, decide the most appropriate priority level based on the impact and urgency.
# Finally, return only the structured output in valid JSON format, like this:
# {"priority": "High"}
# Do not include your reasoning in the output — just the JSON.
# """

# response_prompt = """
# You are provided with a support ticket's text along with its Category, Tags, and assigned Priority level.

# Follow these steps before generating your final response:

# 1. Analyze the ticket text to understand the customer's sentiment and main concern.
# 2. Identify the issue type using the provided Category and Tags.
# 3. Determine the appropriate ETA based on the Priority level.
# 4. Compose a short, empathetic response that reassures the customer, acknowledges their concern, and includes the ETA.

# Ensure the final response:

# 1. Is under 50 words
# 2. Has a polite and empathetic tone
# 3. Addresses the issue clearly

# Return only the final response to the customer. Do not include your reasoning steps in the output.
# """


# # Streamlit App
# st.title("Support Ticket Categorization System")

# st.write("Enter the support ticket text below:")

# support_ticket_input = st.text_area("Support Ticket Text", height=200)

# if st.button("Process Ticket"):
#     if support_ticket_input:
#         st.write("Processing...")

#         # Categorization
#         category_result = classify_ticket(classification_prompt, support_ticket_input)
#         category = category_result.get('Category') if category_result else "N/A"
#         st.subheader("Category:")
#         st.write(category)

#         # Metadata Extraction
#         metadata_result = extract_metadata(metadata_prompt, support_ticket_input)
#         device = metadata_result.get('Device') if metadata_result else "N/A"
#         problem_type = metadata_result.get('Problem Type') if metadata_result else "N/A"
#         user_impact = metadata_result.get('User Impact') if metadata_result else "N/A"

#         st.subheader("Metadata:")
#         st.write(f"Device: {device}")
#         st.write(f"Problem Type: {problem_type}")
#         st.write(f"User Impact: {user_impact}")

#         # Priority Prediction
#         priority_result = predict_priority(priority_prompt, support_ticket_input, problem_type, user_impact)
#         priority = priority_result.get('priority') if priority_result else "N/A"
#         st.subheader("Priority:")
#         st.write(priority)

#         # Draft Response Generation
#         draft_response = generate_response(response_prompt, support_ticket_input, category, metadata_result, priority)
#         st.subheader("Draft Response:")
#         st.write(draft_response)


#     else:
#         st.warning("Please enter support ticket text to process.")


### Docker File

In [ ]:
# %%writefile Dockerfile

# FROM python:3.9-slim

# WORKDIR /app

# RUN apt-get update && apt-get install -y \
#     build-essential \
#     curl \
#     software-properties-common \
#     git \
#     && rm -rf /var/lib/apt/lists/*

# COPY requirements.txt ./
# COPY app.py ./
# COPY src/ ./src/

# RUN pip3 install -r requirements.txt

# EXPOSE 8501

# HEALTHCHECK CMD curl --fail http://localhost:8501/_stcore/health

# ENTRYPOINT ["streamlit", "run", "app.py", "--server.port=8501", "--server.address=0.0.0.0"]


### requirements.txt

In [ ]:
# %%writefile requirements.txt
# altair==5.5.0
# pandas==2.2.2
# streamlit==1.47.1
# openai==1.97.1

## Hugging Face Setup

#### 1. Login to Hugging Face

Go to [Hugging face](https://huggingface.co) and sign up or log in to your account.

#### 2. Create a New Space

   * Navigate to [Hugging face Spaces ](https://huggingface.co/spaces).
   * Click **Create New Space**.
   * Fill in:

     * Name for your Space.
     * Space SDK: Select **Docker**.
     * Choose a Docker template: Select **Streamlit**
     * Visibility: Choose *Public* or *Private*.
   * Click **Create Space**.

#### 3. Setup OpenAI tokens




- Open your Hugging Face Space

- Click on the **“Settings”** tab at the top of the Space.

- Scroll down to the **“Repository secrets”** section.

- Click **“Add a new secret”**.

- In the **Name** field, type token name

- In the **Secret** field, paste your secret key

- Click **“Add secret”** to save it.


#### 4. Upload Your Files




   * In the new Space, click the **Files** tab.
   * Delete existing requirements.txt , DockerFile
   * Click Contribute then Upload files and add:

     * `app.py`
     * `requirements.txt`
     * `DockerFile`
   * Commit the upload.

#### 5. Build and Launch



   * Hugging Face will automatically detect the `Dockerfile` and start building the container.
   * Wait a few minutes for the build to complete and the app to go live.

#### 6. Access and Test

* Go to the App tab or the Space URL to view and test your running Streamlit app.



# Conclusion

The primary objective here was to demonstrate how we can convert unstructured support ticket text into structured information using large language models and prompt engineering.  

By classifying tickets into categories, assigning priorities, and generating structured outputs in JSON format, we’ve created a scalable and reusable system that can significantly streamline customer support operations.


**Business Impact**

- **Improved Efficiency:** Automating ticket classification and prioritization helps support teams triage issues faster, reducing resolution time.
- **Better Resource Allocation:** Tickets can be routed based on urgency and category, ensuring critical issues are addressed first.
- **Data-Driven Insights:** Structured data enables better tracking, reporting, and trend analysis.

This approach serves as a strong foundation for building intelligent customer support systems that are adaptable, efficient, and business-aligned.

**Improvement Areas**

- The current system lacks access to the latest company policies and procedural updates, making responses generic or outdated. RAG addresses this by retrieving the most relevant, up-to-date internal documents to generate accurate and context-aware replies.


<font size=6 color='#4682B4'>Power Ahead</font>
___